# Battle of The Neighborhoods

In this notebook I will be fetching location data using an API and using the data I will build a model to successfully perform clustering of neighborhoods with which are similar to eachother.

In [9]:
import pandas as pd
import numpy as np
print("Hello capstone project course!")

Hello capstone project course!


# 1) Webscraping and extracting neighborhood data
Import required libraries.

In [10]:
from bs4 import BeautifulSoup
import requests
import csv

In [11]:
source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'lxml')

Parsing the required table from the website and converting it into a pandas dataframe.

In [12]:
stuff = soup.find('div', class_='mw-parser-output').table
data = pd.DataFrame()
rowi = 0
for row in stuff.findAll('tr'):
    coli = 0
    for col in row.findAll('td'):
        data.loc[rowi,coli] = col.get_text()
        coli += 1
    rowi += 1

data.columns = ['PostCode', 'Borough', 'Neighborhood']
data['Neighborhood'] = pd.Series(data['Neighborhood']).str.rstrip('\n')

Here I am removing data values where Borough has not been assgined.

In [13]:
data.drop(data[data['Borough'] == 'Not assigned'].index, inplace = True)
data.reset_index(drop = True, inplace = True)

Grouping Neighborhoods with same postal code and replaceing neighborhoods with borough values where it has not been assigned.

In [14]:
df = data.groupby(['PostCode'], as_index = False).agg({'Borough' : 'max', 'Neighborhood': ','.join}, inplace = True)
df['Neighborhood'] = np.where(df['Neighborhood'] == 'Not assigned', df['Borough'], df['Neighborhood'])
df.head()

,PostCode,Borough,Neighborhood
0,M1B,Scarborough,"Rouge,Malvern"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Guildwood,Morningside,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [15]:
df.shape

(103, 3)

In [16]:
!pip install geocoder
import geocoder
postal_code1 = df['PostCode']
long = []
lat = []
for code in postal_code1:
    g = geocoder.arcgis('{}, Toronto, Ontario'.format(code))
    lat_lng_coords = g.latlng
    long.append(lat_lng_coords[1])
    lat.append(lat_lng_coords[0])

     |████████████████████████████████| 102kB 16.5MB/s ta 0:00:01


In [17]:
long_series = pd.Series(long)
lat_series = pd.Series(lat)
frame = {'PostCode': postal_code1 , 'Latitude': lat_series, 'Longitude': long_series}
df1 = pd.DataFrame(frame)
result = pd.merge(df,df1, on = 'PostCode' )
result.head()

,PostCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.811525,-79.195517
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.785665,-79.158725
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.765815,-79.175193
3,M1G,Scarborough,Woburn,43.768369,-79.217590
4,M1H,Scarborough,Cedarbrae,43.769688,-79.239440


### Getting coordinates for Toronto, CA 

In [18]:
from geopy.geocoders import Nominatim
address = 'Toronto, CA'
geolocator = Nominatim(user_agent = 'ca_agent')
ngb_latlong = geolocator.geocode(address)
ngb_lat = ngb_latlong.latitude
ngb_long = ngb_latlong.longitude

### Creating a map of toronto using the folium library

In [19]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    altair-4.0.1               |             py_0         575 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.0 MB

The following NEW packages will be 

## Plotting neighborhoods on the map of Toronto

In [20]:
toronto_map =  folium.Map(location = [ngb_lat, ngb_long], zoom_start = 11)
for lat,lng,borough,neighborhood in zip(result['Latitude'], result['Longitude'], result['Borough'], result['Neighborhood']):
    label = '{}, {}'.format(neighborhood,borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
    [lat,lng],
    radius = 5,
    popup = label,
    color = 'blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(toronto_map)  
toronto_map

Here I am filtering out the bouroughs not containing 'Toronto'

In [21]:
filtered_df = result[result['Borough'].str.contains('Toronto')].reset_index(drop = True)

#### Define Foursquare Credentials and Version

In [22]:
CLIENT_ID = 'XGTTY3F1R2SG4JQ4BQNN5KUQK0HNKOUWRKSYPGO3Q3OMW1DI' # your Foursquare ID
CLIENT_SECRET = 'B4ZG32CPPGC4M1GBMC50XWY1W3WS2GRUMMOQ2NZMYZQRWFVI' # your Foursquare Secret
VERSION = '20200131' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XGTTY3F1R2SG4JQ4BQNN5KUQK0HNKOUWRKSYPGO3Q3OMW1DI
CLIENT_SECRET:B4ZG32CPPGC4M1GBMC50XWY1W3WS2GRUMMOQ2NZMYZQRWFVI


### Writing a function to get nearby venues of all neighbhorhoods

In [23]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [24]:
toronto_venues = getNearbyVenues(names=filtered_df['Neighborhood'],
                                   latitudes=filtered_df['Latitude'],
                                   longitudes=filtered_df['Longitude']
                                  )

The Beaches
The Danforth West,Riverdale
The Beaches West,India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park,Summerhill East
Deer Park,Forest Hill SE,Rathnelly,South Hill,Summerhill West
Rosedale
Cabbagetown,St. James Town
Church and Wellesley
Harbourfront
Ryerson,Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide,King,Richmond
Harbourfront East,Toronto Islands,Union Station
Design Exchange,Toronto Dominion Centre
Commerce Court,Victoria Hotel
Roselawn
Forest Hill North,Forest Hill West
The Annex,North Midtown,Yorkville
Harbord,University of Toronto
Chinatown,Grange Park,Kensington Market
CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place,Underground city
Christie
Dovercourt Village,Dufferin
Little Portugal,Trinity
Brockton,Exhibition Place,Parkdale Village
High Park,The Junction South
Parkdale,Roncesvalles
Runnymede

Let's check how many venues were returned for each neighborhood

In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide,King,Richmond",100,100,100,100,100,100
Berczy Park,61,61,61,61,61,61
"Brockton,Exhibition Place,Parkdale Village",69,69,69,69,69,69
Business Reply Mail Processing Centre 969 Eastern,100,100,100,100,100,100
"CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara",71,71,71,71,71,71
"Cabbagetown,St. James Town",40,40,40,40,40,40
Central Bay Street,95,95,95,95,95,95
"Chinatown,Grange Park,Kensington Market",75,75,75,75,75,75
Christie,11,11,11,11,11,11


#### Let's find out how many unique categories can be curated from all the returned venues

In [26]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 221 unique categories.


## 3. Analyze Each Neighborhood

In [27]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Afghan Restaurant,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,...,Trail,Train Station,Tram Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,"Adelaide,King,Richmond",0.000000,0.0000,0.030000,0.000000,0.010000,0.000000,0.030000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00
1,Berczy Park,0.000000,0.0000,0.000000,0.000000,0.016393,0.000000,0.000000,0.000000,0.016393,...,0.00000,0.000000,0.00000,0.016393,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
2,"Brockton,Exhibition Place,Parkdale Village",0.000000,0.0000,0.000000,0.000000,0.014493,0.014493,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.028986,0.000000,0.014493,0.000000,0.000000,0.000000,0.00
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.0000,0.020000,0.000000,0.000000,0.010000,0.020000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.020000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",0.014085,0.0000,0.000000,0.000000,0.000000,0.000000,0.014085,0.000000,0.000000,...,0.00000,0.014085,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
5,"Cabbagetown,St. James Town",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
6,Central Bay Street,0.000000,0.0000,0.010526,0.000000,0.010526,0.000000,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.000000,0.010526,0.010526,0.010526,0.000000,0.000000,0.00
7,"Chinatown,Grange Park,Kensington Market",0.000000,0.0000,0.000000,0.000000,0.040000,0.013333,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.040000,0.000000,0.066667,0.013333,0.000000,0.000000,0.00
8,Christie,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.090909,0.000000,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
9,Church and Wellesley,0.000000,0.0125,0.012500,0.000000,0.000000,0.012500,0.000000,0.000000,0.000000,...,0.00000,0.000000,0.00000,0.000000,0.000000,0.012500,0.000000,0.012500,0.000000,0.00


#### Let's print each neighborhood along with the top 5 most common venues

In [29]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide,King,Richmond----
         venue  freq
0  Coffee Shop  0.07
1         Café  0.06
2        Hotel  0.04
3   Steakhouse  0.04
4   Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.08
1        Cocktail Bar  0.05
2      Farmers Market  0.03
3  Seafood Restaurant  0.03
4                Café  0.03


----Brockton,Exhibition Place,Parkdale Village----
                    venue  freq
0             Coffee Shop  0.09
1                    Café  0.07
2  Furniture / Home Store  0.06
3              Restaurant  0.06
4                  Bakery  0.04


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0         Coffee Shop  0.09
1          Steakhouse  0.04
2               Hotel  0.04
3                Café  0.03
4  Seafood Restaurant  0.03


----CN Tower,Bathurst Quay,Island airport,Harbourfront West,King and Spadina,Railway Lands,South Niagara----
                venue  freq
0         Coffee Shop  0.10
1  Italian Re

### Put the top 10 venues of each neighbhorhood into a dataframe

In [30]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide,King,Richmond",Coffee Shop,Café,Hotel,Steakhouse,Gastropub,Bakery,Breakfast Spot,American Restaurant,Restaurant,Asian Restaurant
1,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Bakery,Steakhouse,Farmers Market,Seafood Restaurant,Restaurant,Breakfast Spot,Café
2,"Brockton,Exhibition Place,Parkdale Village",Coffee Shop,Café,Furniture / Home Store,Restaurant,Bakery,Sandwich Place,Bar,Italian Restaurant,Gym,Hotel
3,Business Reply Mail Processing Centre 969 Eastern,Coffee Shop,Steakhouse,Hotel,Gym,Sushi Restaurant,Bar,Café,Seafood Restaurant,Pub,Thai Restaurant
4,"CN Tower,Bathurst Quay,Island airport,Harbourf...",Coffee Shop,Italian Restaurant,Café,Bar,Park,Restaurant,Pub,Speakeasy,Bakery,Electronics Store


## 4. Cluster Neighborhoods and Visualize the clusters on Toronto's map

In [32]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int32)

In [33]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = filtered_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head() # check the last columns!

,PostCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676531,-79.295425,0.0,Health Food Store,Pub,Trail,Donut Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
1,M4K,East Toronto,"The Danforth West,Riverdale",43.683178,-79.355105,1.0,Bus Line,Grocery Store,Park,Discount Store,Women's Store,Eastern European Restaurant,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market
2,M4L,East Toronto,"The Beaches West,India Bazaar",43.667965,-79.314667,0.0,Park,Sandwich Place,Gym,Italian Restaurant,Pizza Place,Pub,Movie Theater,Fast Food Restaurant,Fish & Chips Shop,Burrito Place
3,M4M,East Toronto,Studio District,43.660629,-79.334855,0.0,Diner,Brewery,Italian Restaurant,Pizza Place,Sushi Restaurant,Café,Sandwich Place,Bar,Gastropub,Coffee Shop
4,M4N,Central Toronto,Lawrence Park,43.728420,-79.387133,3.0,Bus Line,Swim School,Women's Store,Flower Shop,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant


In [34]:
toronto_merged['Cluster Labels'].dropna(inplace = True)

In [35]:
import matplotlib.cm as cm
import matplotlib.colors as colors
# create map
map_clusters = folium.Map(location=[ngb_lat, ngb_long], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster)-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters